# MosesAI Notebook: OpenAI v1.x + LangChain + Pinecone v3.x

In [1]:
# ✅ Prerequisites (run this once)
# pip install openai langchain langchain-openai langchain-pinecone pinecone-client python-dotenv

  Using cached pinecone_client-6.0.0-py3-none-any.whl.metadata (3.4 kB)
Using cached pinecone_client-6.0.0-py3-none-any.whl (6.7 kB)


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

Exception: The official Pinecone python package has been renamed from `pinecone-client` to `pinecone`. Please remove `pinecone-client` from your project dependencies and add `pinecone` instead. See the README at https://github.com/pinecone-io/pinecone-python-client for more information on using the python SDK.

In [ ]:
# 1. Load environment variables
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [ ]:
# 2. Initialize clients
client = OpenAI(api_key=OPENAI_API_KEY)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "talmud-pages"
pc_index = pc.Index(index_name)

In [ ]:
# 3. Wrap Pinecone index with LangChain interface
index = PineconeVectorStore(index=pc_index, embedding=embeddings)

In [ ]:
# 4. Define search function
def get_similar_docs(query, k=5, score=False):
    if score:
        return index.similarity_search_with_score(query, k=k)
    return index.similarity_search(query, k=k)

In [ ]:
# 5. Run a sample query
query = "When do you say Shema?"
similar_docs = get_similar_docs(query)

print(f"Top {len(similar_docs)} results for query: '{query}'\n")
for i, doc in enumerate(similar_docs, 1):
    if isinstance(doc, tuple):
        doc, score = doc
        print(f"Result {i} (score={score:.2f}):\n{doc.page_content[:300]}\n")
    else:
        print(f"Result {i}:\n{doc.page_content[:300]}\n")